In [1]:
import json, os
from openai import OpenAI

source_data_path = ""
filename = "summary_Russo_Ukrainian_War_eng.json"
source_filename = source_data_path + filename

des_data_path = ""
des_filename = des_data_path + "v3_auto_QnA_" + filename

# Load articles
with open(source_filename, "r", encoding="utf-8") as f:
    articles = json.load(f)

# Load questions
with open(
    des_data_path + "formated_output_questions_v3.json", "r", encoding="utf-8"
) as f:
    questions = json.load(f)

# Set API key
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [2]:
def get_answers(article, questions, model):
    article_title = article.get("title")
    article_content = article.get("body")
    print(f"Proceeding article {article_title}")
    # answers = {}
    # for q_id, question in questions.items():

    pre = "You are an AI trained to understand articles and generate concise answers to multiple-choice questions based on the content. \
        Please read the following article carefully. After reading, answer ALL the questions listed below. \
            Your answers must be in capital letters and formatted as a single string, where each question number is followed by its corresponding answer letter. \
                Separate each question-answer pair with a semicolon. \
                    Example format: 1A;2B;3C;4D;... \n\n"

    prompt = pre + f"Article Content:\n{article_content}\n\n Questions: {questions}\n"
    try:
        response = client.chat.completions.create(
            model=model, messages=[{"role": "user", "content": prompt}], temperature=0
        )
        # content = response.choices[0].message.content
        # answers[q_id] = content
    except Exception as e:
        print(f"Error: {e}")
        # answers[q_id] = "Sorry, error from GPT."
    return response

In [3]:
def convert_string_to_dict(answer_string):
    if answer_string == "":
        return {}
    if answer_string[-1] == ";":
        answer_string = answer_string[:-1]
    return {item[:-1]: item[-1] for item in answer_string.split(";")}


results = []
iteration = 0
max_iteration = len(articles)

for article in articles:
    # article_content = article['body']

    answers = get_answers(article, questions, model="gpt-4o")

    pure_answers = answers.choices[0].message.content
    print(pure_answers)

    # convert string to dictionary 1A;2B;3C;44D to {'1': 'A', '2': 'B', '3': 'C', '44': 'D'}
    processed_answers = convert_string_to_dict(pure_answers)
    article["answers"] = processed_answers

    results.append(article)
    iteration += 1
    print(f"Processed {iteration} out of {max_iteration}")


# Save results to a JSON file
with open(des_filename, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

Proceeding article PM Trudeau travels to Washington for NATO Summit - Caribbean News Global
1A;2D;3C;4E;5C;6A;7B;8B
Processed 1 out of 100
Proceeding article NATO will strive to project unity at Washington summit, but cracks emerge
1A;2B;3B;4E;5C;6D;7B;8B
Processed 2 out of 100
Proceeding article U.S. Unveils Major New Military Aid for Ukraine
1A;2B;3A;4B;5B;6A;7C;8A
Processed 3 out of 100
Proceeding article US provides major boost to Ukraine's Patriot system
1A;2B;3A;4B;5B;6A;7B;8A
Processed 4 out of 100
Proceeding article HOW THE U.S. EMPIRE'S WAR AGAINST CHINA IS BEING WAGED IN MYANMAR (BURMA) - Russia News Now
1E;2E;3E;4E;5E;6E;7E;8E
Processed 5 out of 100
Proceeding article AI begins ushering in an age of killer robots  - NZ Herald
1E;2E;3B;4B;5B;6E;7A;8E
Processed 6 out of 100
Proceeding article Electric Toothbrush Market Size to Touch USD 7.47 billion, Rising at a CAGR of 7.9% by 2031
1E;2E;3E;4E;5E;6E;7E;8E
Processed 7 out of 100
Proceeding article Industrial Barrier Systems Ma